In [1]:
import pandas as pd
import lxml
import html5lib

In [64]:
import datetime

In [48]:
# download data
weather_tables = pd.read_html("http://w1.weather.gov/obhistory/KCLT.html")

In [174]:
# column headings
col_labels="""
day
time
wind_mph
vis_mi
weather
sky
temperature_f
dewpoint_f
temp_6hr_max
temp_6hr_min
humidity
wind_chill_f
heat_index_f
pressure_in
pressure_mb
precipitation
precipitation_3hr
precipitation_6hr
"""
col_labels = col_labels.strip().split("\n")

In [175]:
# fix columns
data = weather_tables[3].copy()

# column names
data.columns = col_labels
data.drop(['temp_6hr_max', 'temp_6hr_min', 'precipitation_3hr', 'precipitation_6hr'], axis=1, inplace=True)

In [176]:
# remove the header and footer
# this should be the top 3 and bottom 3 rows. But here I'll look for any rows that don't have an integer
# in the day column
def is_int(x):
    try:
        int(x)
        return True
    except:
        return False
i_is_int = (data.day.apply(is_int))
data = data[i_is_int]

In [177]:
# remove % from humidity
data.humidity = data.humidity.apply(lambda x: x.replace("%", ""))

In [178]:
# make a date_time column
data['month'] = datetime.datetime.today().month
data['year'] = datetime.datetime.today().year
data['datetime'] = data.month.apply(str) + "/" + data.day.apply(str) + "/" + data.year.apply(str) \
    + " " + data.time.apply(str)
data['datetime'] = pd.to_datetime(data.datetime)

data.drop(['day', 'month', 'year', 'time'], axis=1, inplace=True)
cols = ['datetime'] + [_ for _ in data.columns if _ is not 'datetime']
data = data[cols]
#data.set_index('datetime', inplace=True)

In [180]:
# convert columns to numeric
for col in ['vis_mi', 'temperature_f', 'dewpoint_f', 'humidity', 'wind_chill_f', 'heat_index_f',
            'pressure_in', 'pressure_mb', 'precipitation']:
    data[col] = pd.to_numeric(data[col], errors='raise')

In [181]:
data.head()

,datetime,wind_mph,vis_mi,weather,sky,temperature_f,dewpoint_f,humidity,wind_chill_f,heat_index_f,pressure_in,pressure_mb,precipitation
3,2016-04-06 11:52:00,Vrbl 6,10,Partly Cloudy,SCT250,51,21,31,NaN,NaN,30.35,1027.8,NaN
4,2016-04-06 10:52:00,SE 6,10,A Few Clouds,FEW250,48,20,33,45,NaN,30.38,1028.7,NaN
5,2016-04-06 09:52:00,S 7,10,A Few Clouds,FEW250,45,20,37,41,NaN,30.38,1028.6,NaN
6,2016-04-06 08:52:00,S 9,10,A Few Clouds,FEW250,42,22,45,37,NaN,30.37,1028.4,NaN
7,2016-04-06 07:52:00,SE 8,10,A Few Clouds,FEW250,38,23,55,32,NaN,30.36,1028.4,NaN


# Create database

In [103]:
import sqlite3

In [139]:
db = sqlite3.connect("weather.db")
db.row_factory= sqlite3.Row

In [182]:
sql = """
CREATE TABLE readings (
  datetime TIMESTAMP PRIMARY KEY,
  wind_mph TEXT,
  vis_mi NUMERIC,
  weather TEXT,
  sky TEXT,
  temperature_f NUMERIC,
  dewpoint_f NUMERIC,
  humidity NUMERIC,
  wind_chill_f NUMERIC,
  heat_index_f NUMERIC,
  pressure_in NUMERIC,
  pressure_mb NUMERIC,
  precipitation NUMERIC
)
"""
db.execute("DROP TABLE IF EXISTS readings")
db.execute(sql)
db.commit()

# Append to database

In [191]:
import IPython.display

In [200]:
added = 0
duplicated = 0
for i,row in data.iterrows():
    row = row.to_frame().T
    row.set_index('datetime', inplace=True)
    #IPython.display.display(row)
    try:
        row.to_sql("readings", db, if_exists='append')
        added += 1
    except sqlite3.IntegrityError:
        duplicated += 1        

db.commit()
print "{} added, {} duplicated".format(added, duplicated)

0 added, 72 duplicated
